In [ ]:
foldername = ["20230708", "20230709", "20230723", "20230816"][2]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ccdproc
from ccdproc import ImageFileCollection

from astropy.nddata import CCDData
import arya

In [ ]:
from convenience_functions import show_image, combine_images, show_image_residual, mad_std

In [ ]:
def get_flats(foldername, filt):
    imgfiles = ImageFileCollection(foldername, glob_include=f"flat_{filt}_*.fits")

    return imgfiles

In [ ]:
filts = ["g", "r", "i"]

In [ ]:
imgfiles = {filt: get_flats(foldername + "/unbiased/", filt) for filt in filts}

In [ ]:
flats_stacked = {filt: CCDData.read(foldername + f"/flat_{filt}_stacked.fits") for filt in filts}

In [ ]:
flat_std = {filt: CCDData.read(foldername + f"/flat_{filt}_stacked_err.fits") for filt in filts}

In [ ]:
flat2 = {filt: np.median(np.stack([x / np.median(x) for x in imgfiles[filt].data()]), axis=0)
            for filt in filts}

In [ ]:
noise = np.mean([np.median(flat_std[filt]) for filt in filts])
noise

# Plots

In [ ]:
px_max = {
    "g": 1.2,
    "r": 1.2,
    "i": 1.40
}

In [ ]:
Npix = 1797 * 2003

In [ ]:
def plot_noise_hist(Npix=Npix, ymin=1):
    x = 1 + np.linspace(-0.1, 0.1, 1000)
    y = Npix * 1 / np.sqrt(2*np.pi * noise**2) * np.exp(-(x-1)**2 / (2 * noise**2))

    plt.plot(x[y>ymin], y[y>ymin], color="red")

In [ ]:
for filt in filts:
    print(filt)
    for (img, name) in imgfiles[filt].ccds(return_fname=True):
        show_image(img, dpi=100, figsize=(3, 3))
        plt.title(name)


In [ ]:
def flat_scale(img):
    return 1/np.median(img.data)

In [ ]:
for filt in filts:
    for img, fname in imgfiles[filt].ccds( return_fname=True):
        flat = flats_stacked[filt]
        img_reduced = img.data * flat_scale(img) / flat
        
        show_image(img_reduced, clabel = "flat / flat mean")

        plt.tight_layout()
        # savefig("flat_residual." + fname)

# Comparison between different flat methods

In [ ]:
from trim_images import trim_image

In [ ]:
flats_master = {filt: trim_image(CCDData.read(f"master_calibration/masterflat{filt.upper()}.fits", unit="adu")) for filt in filts}

In [ ]:
for filt in filts:
    show_image(flats_stacked[filt].data / flats_master[filt].data * np.median(flats_master[filt]))

# Outlier management

In [ ]:
for filt in filts:
    print(np.median(px_maxima[filt]))


In [ ]:
px_maxima = {
    "g": [],
    "r": [],
    "i": []
}

px_3sigma = {
    "g": [],
    "r": [],
    "i": []
}

for filt in filts:
    print(np.median(px_maxima[filt]))

    for foldername in ["20230708", "20230709", "20230723", "20230816"]:
        imgfiles = get_flats(foldername + "/unbiased/", filt) 
        for img in imgfiles.ccds():
            px_maxima[filt].append(np.quantile(img / np.median(img),  1- 2.8665157187919365e-7))
            px_3sigma[filt].append(np.quantile(img / np.median(img),  1- 0.001349898031630093))
            

# Noise characteristics

In [ ]:
for filt in filts:
    print(np.median(flat_std[filt] / flats_stacked[filt].data))

In [ ]:
for filt in filts:
    print(np.median(flat_std[filt] / flats_stacked[filt].data))
    show_image(flat_std[filt] / flats_stacked[filt].data, clabel=r"$\delta$ value / value")